## Symmetric Key Cryptography

*SKC* is the most intuitive form of cryptography. With SKC, confidential information is secured via symmetric key encryption (SKE), that is, by using a *single secret key* for both encryption and decryption. 

SKC involves: 
- an encryption function that converts a given plain text to a *ciphertext* while utilizing a secret key
- a decryption function that inverts the operation by converting the ciphertext back to plain text using the same secret key


### Properties of Symmetric Key Cryptosystems

A symmetric key cryptosystem ensures the following properties: 

- Confidentiality: refers to the property that the information content of the encrypted messages is protected from unauthorized access
- Integrity; refers to the property that any tampering of encryptd messages during storage or transmission can be detected
- Authenticity: refers to the property that the receiver of a message can verify the identity of the sender

These properties should be realized in a setting where the algorithms used for encryption + decryption may be public and where access to the information content of encrpyted messages is controlled exclusively through access to the secret key. 

#### Two Main Tasks

1. Employing a robust symmetric key encryption algorithm resistant to cryptographic attacks 
2. Ensuring confidentiality in the distribution and management of secret keys

## Advanced Encryption Standard (AES) Cipher

In [9]:
# Set the plaintext we want to encrypt
plaintext=u"this is a strict top secret message for intended recipients only"
print(f"\nGiven plaintext: {plaintext}")


Given plaintext: this is a strict top secret message for intended recipients only


We start by creating the key, in this case, a random 16-letter string. 

In [6]:
from secretpy import Caesar
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from functools import reduce
import numpy as np

alphabet=('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ')

# lamba defines an inline function in this case that takes two values a,b with the resulting expression of a+b
# reduce uses a two-argument function(above), and applies this to all the entries in the list (random alphabet characters) cumulatively
aes_key = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])

print(f'AES secret key: {aes_key}')

AES secret key: nzhkhhqtklwvliak


AES supports multiple operating modes. We choose the *Cipher Block Chaining (CBC) mode*. This requires specifying a random *Initialization Vector (IV)* also called a *nonce*. 

In [7]:
aes_initialization_vector = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])
print(f"AES initialization vector: {aes_initialization_vector}")

AES initialization vector: iconhzdtouysahoy


In [10]:
# The encryptor is setup using the key & CBC. In both cases we need to convert the string (utf-8) into bytes
sender_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_encryptor = sender_aes_cipher.encryptor()

# update can add text to encypt in chunks, and then finalize is needed to complete the encryption process
aes_ciphertext = aes_encryptor.update(bytes(plaintext, 'utf-8')) + aes_encryptor.finalize()

# Note the output is a string of bytes
print(f"Encrypted AES ciphertext: {aes_ciphertext}")

Encrypted AES ciphertext: b"w\xd2\x1e\x80\x11\xe4B\x89\xe5R\x9d\x1eQ\x12\xfb\x02\xa7\x92\xc9T\xfb\x84\xb3\xf7\x93!j{\x9ayd\xce\xd9\x85X\x7f\x1f\xf0i\xe9\x89\xd5\x1bp\xe3\xb3\xf5ry\\\xc8\x1e\x97\x87D\n'\x1c\x9a\x7f\x81\x8fB@"


To decrypt it, let's instantiant the AES cipher on behalf of the reciever. Note that the intended reciever has access to both the secret key and the initialization vector. But the latter doesn't have to be secret. 

In [11]:
# Similar setup of AES to what we did for encryption, but this time, for decryption
receiver_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_decryptor = receiver_aes_cipher.decryptor()

# Do the decryption
aes_plaintext_bytes = aes_decryptor.update(aes_ciphertext) + aes_decryptor.finalize()

# convert back to a character string (we assume utf-8)
aes_plaintext = aes_plaintext_bytes.decode('utf-8')

print(f"Decrypted AES plaintext: {aes_plaintext}")

Decrypted AES plaintext: this is a strict top secret message for intended recipients only


#### Block Ciphers

*Block Ciphers* like DES or AES -- achieve security by combining the principles of *confusion* and *diffusion*. 

- Confusion: The characteristic where each bit in the ciphertext depends on multiple bits of the secret key. It ensures that a small change in the secret key modifies almost all the bits of the cipher text. 

- Diffusion: The characteristic where flipping a single bit in the plain text should modify roughly half the bits in the cipher text and vice versa. Diffusion hides the statistical relationship between the message and the cipher text. Ciphers with adequate diffusion satisfy the *avalanche criteria* of cryptography. 

Block ciphers implement confusion diffusion using cryptographic structures known as *substitution-permutation networks (SPNs)* operating on discrete blocks of data. 

An SPN accepts a block of plain text and the secret key and performs a number of *rounds* of transformations to yield a ciphertext block. Each round is composed of alternating mathematical structures known as substitution boxes (S-boxes) and permutation boxes (P-boxes), or equivalent operations. 

These respectively implement complex nonlinear and linear transformations on the input blocks -- leading to the avalanche effect.

#### Advantages of Symmetric Key Cryptopgraphy

1. Speed & Efficiency: SKC algorithms are more suitable for larger volumes of data or fore real-time communication since they are generally less resource-intensive than asymmetric key encryption.

2. Scalability: SKC algorithms, because of their low computational requirement, can scale well with the number of users and the amount of data being encrypted. 

3. Simplicity: They are easier to implement and understand relative to asymmetric approaches.


#### Challenges of Symmetric Key Cryptopgraphy

1. Key Distribution and Management: Both the sender and receiver of a message must have access to the same key, which must be kept confidential. 

2. Lack of non-repudiation: Non-repudiation refers to the abilitiy to prove that a specific party has sent a message. In SKC, since the same key is used for encryption and decryption, its not possible to determine which party created a specific cipher text. 

Asymmetric encryption solves non-repudiation via the use of *digital signatures*.


### Quantum Computing & SKC: Risks and Mitigation

#### Quantum Cryptographic Attacks

There are two distinct classes of quantum threats to traditional cryptographic algorithms: 

1. Quantum **brute force** attacks: Where the attacker uses a quantum computer to execute a specialized quantum algorithm to conduct a brute force search through the key space of a symmetric cipher. Like **Grover's Algorithm**. 

2. Quantum **cryptanalytic** attacks: These refer to situations where quantum computers are deployed to execute cryptanalytic attacks that aim to recover either the secret key or plian text in a more efficient manner than brute force. 

#### Quantum Risk Mitigation

Typically security level is expressed in bits. $N$-bit security means it requires $O(2^N)$ operations to break. For classical computers, the security level is roughly synonymous with the key length. 

##### Brute Force Attacks 

A quantum brute force attack changes the above assessment because Grover's algorithm enables an attacker with a suitable quantum computer to search the key space of a cipher quadratically faster than any classical computer.

For instance breaking the $2^{128}$ bit encryption for AES-128 can be solved with just $2^{64}$ for a suitable quantum computer. A security level of 64 bits is already conventionally deemed insecure. 

The obvious way to mitigate brute force attacks is to double the minimum key lengths used for symmetric encryption. However this implies incredible additional computational cost for routine encryption decryption tasks, with slower performance, more memory requirement, and additinoal energy use. 

##### Cryptanalytic attacks and mitigation 

This is currently being actively researched. The combination of classical and quantum techniques of attack expands the array of tools available to attackers to probe weaknesses in the mathematical structure of ciphers.